## Preprocessing the data

- 시계열 데이터를 회귀 문제로 다루기 위해 시간 관련 변수들을 따로 생성

- 건물, 요일, 시간에 따라 유의미한 전력사용량 차이가 있기 때문에, 파생변수 추가

- 평일과 주말&공휴일의 전력사용량 차이가 있기 때문에 이를 0과 1로 구분

- 시간의 순환적 성격을 반영하기 위해 sin_time와 cos_time 변수 생성. 23시와 00시는 시간적으로 매우 가깝지만 모델은 멀리 떨어져 있다고 판단할 수 있음

- 전력사용량은 절대적인 지표보다 사람이 느끼는 정도에 따라 달라질 수 있음. 따라서 불쾌지수 변수(THI)와 체감온도(sensory_temp) 변수를 생성

- CDH(Cooling Degree Hour) : 냉방을 위한 필요 에너지를 산정하는 데 쓰는 변수

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
train = pd.read_csv('./data/train.csv')
train.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


- 아래와 같이 정의하면, 삼각함수의 경우 같은 값이 나옴
    - 그래서 둘 다 정의한 것 아닐까?
    - 순환적 성격을 반영하기 위해서는, 같은 것이 나올 수 밖에 없음
    - 두 변수를 하나로 묶어서 처리하면 더 효율적이지 않을까?
```
    train['sin_time'] = np.sin(2*np.pi*train.hour/24)
    train['cos_time'] = np.cos(2*np.pi*train.hour/24)
```
- 주말 : 대기전력 -> 평일 전력 - 주말 전력 = 사람이 사용하는 전력 -> 효율적인 학습 가능?
    - 주말에 사람이 없는 건물을 대상으로!

In [ ]:
## 데이터 불러온 후 num_date_time 열 삭제
train = pd.read_csv('./data/train.csv')
train = train.drop("num_date_time", axis = 1)

## 변수들을 영문명으로 변경
cols = ["num", "date_time", "temp", "rain", "wind", "hum", "sun_hr", "sun_amount", "power"]
train.columns = cols

## 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.weekofyear

#######################################
## 건물별, 요일별, 시간별 발전량 평균 넣어주기
#######################################
power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_hour_mean'] = train.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 시간별 발전량 평균 넣어주기
#######################################
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean'] = train.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 시간별 발전량 표준편차 넣어주기
#######################################
power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std'] = train.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 요일별 발전량 평균 넣어주기
#######################################
power_day_mean = pd.pivot_table(train, values = 'power', index = ['num', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_mean'] = train.progress_apply(lambda x : power_day_mean.loc[(power_day_mean.num == x['num']) & (power_day_mean.day == x['day']) ,'power'].values[0], axis = 1)

#######################################
## 건물별 요일별 발전량 표준편차 넣어주기
#######################################
power_day_std = pd.pivot_table(train, values = 'power', index = ['num', 'day'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['day_std'] = train.progress_apply(lambda x : power_day_std.loc[(power_day_std.num == x['num']) & (power_day_std.day == x['day']) ,'power'].values[0], axis = 1)

## 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('2020-08-17'<=train.date_time)&(train.date_time<'2020-08-18'), 'holiday'] = 1

## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32

## 체감온도 산출
train['sensory_temp'] = 13.12 + 0.6215*train['temp'] - 11.37*(train['wind']**0.16) + 0.3965*train['temp']*(train['wind']**0.16)

## CDH 변수 추가
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = train[train['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

<ipython-input-4-6fb580f2ff2c>:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train['week'] = date.dt.weekofyear
100%|██████████| 204000/204000 [01:37<00:00, 2095.02it/s]


In [ ]:
## 결측치가 많고, test 데이터에 없는 sun_hr, sun_amount 변수 제외 후 column명 변경
train = train.drop(["sun_hr", "sun_amount"], axis = 1)

## save the preprocessed data
train.to_csv('/content/drive/MyDrive/Colab Notebooks/XGBoost/data/train_preprocessed.csv', index = False)